In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import useful_functions as uf

#load_transform_split(fpath='data/ALL_YEARS_ADDED_FEATURES.csv',
#                         target='DROPOUT_N', expand=False, split=0.1, clean=True,
#                         drop_feats=['SCHOOL_YEAR','DIV_NAME','SCH_NAME','DIPLOMA_RATE'],
#                         fmt='numpy',return_pipeline=False):

X_train_scaled, X_test_scaled, y_train, y_test = uf.load_transform_split(target='DROPOUT_N')

In [2]:
# global variables about whether or not to modify the hyperparameters
USE_BATCH_NORMALIZATION = True
USE_ADAM_OPTIMIZER = True
USE_DROPOUT_REGULARIZATION = True
USE_SPECIAL_INITIALIZATION = True
USE_LR_SCHEDULER = True

# some of the modifications to the hyperparams -- don't really expect these to change
ACTIVATOR = 'selu' # the one suggested by slides 21 & 44 on Lecture 11
INITIALIZER = 'he_normal' # the one suggested by slides 17 & 44 on Lecture 11

# some more modifications to the hyperparams -- probably change these later on
# for now, set them to the default values (except the LR -- upped by a factor of 10)
DROPOUT_RATE = 0.2
ADAM_LR = 0.01
ADAM_BETA1 = 0.9
ADAM_BETA2 = 0.999
NUM_NODES = 10
N_EPOCHS = 30
LR_10FOLD_DECAY_TIME = 20 # decay by a factor of 10 every X epochs

In [3]:
import tensorflow as tf
from tensorflow import keras

def initialize_model(use_drop=USE_DROPOUT_REGULARIZATION, use_batch=USE_BATCH_NORMALIZATION,
                    drop_rate=DROPOUT_RATE, num_nodes=NUM_NODES,
                    use_init=USE_SPECIAL_INITIALIZATION, init=INITIALIZER, activ=ACTIVATOR):
    model = keras.models.Sequential()
    if use_drop:
        model.add(keras.layers.Dropout(rate=drop_rate))
    if use_init:
        model.add(keras.layers.Dense(19, activation=activ, input_shape=X_train_scaled.shape[1:],
                                     kernel_initializer=init))
    else:
        model.add(keras.layers.Dense(19, activation=activ, input_shape=X_train_scaled.shape[1:]))
    if use_batch:
        model.add(keras.layers.BatchNormalization())
        
        
    for i in np.arange(19.0, 1, NUM_NODES/(-18)):
        j = int(i)
        if use_drop:
            model.add(keras.layers.Dropout(rate=drop_rate))
        if use_init:
            model.add(keras.layers.Dense(j, activation=activ, kernel_initializer=init))
        else:
            model.add(keras.layers.Dense(j, activation=activ))
        if use_batch:
            model.add(keras.layers.BatchNormalization())
    if use_init:
        model.add(keras.layers.Dense(1, kernel_initializer=init))
    else:
        model.add(keras.layers.Dense(1))
    return(model)

In [4]:
def compile_model(model, use_adam=USE_ADAM_OPTIMIZER, lr=ADAM_LR, b1=ADAM_BETA1, b2=ADAM_BETA2):
    if use_adam:
        optimizer=keras.optimizers.Adam(learning_rate=lr, beta_1=b1, beta_2=b2)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
    else:
        model.compile(loss='mean_squared_error')
    return(model)

In [5]:
def fit_model(model, use_sched=USE_LR_SCHEDULER, decay_time=LR_10FOLD_DECAY_TIME, init=ADAM_LR,
              use_adam=USE_ADAM_OPTIMIZER, n_epo=N_EPOCHS):
    if not use_adam or not use_sched:
        history=model.fit(X_train_scaled, y_train, epochs=n_epo)
    else:
        def lr_fn(epoch): return(init * 10**(-epoch/decay_time))
        lr_scheduler = keras.callbacks.LearningRateScheduler(lr_fn)
        history = model.fit(X_train_scaled, y_train, epochs=n_epo, callbacks=[lr_scheduler])
    return(history, model)

In [6]:
from sklearn.metrics import mean_squared_error
def analyze_performance(history, model):
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('RMSE: %.2f' %rmse)
    return(rmse)

In [7]:
# can I just do this use *args or something? IDK
def run(use_drop=USE_DROPOUT_REGULARIZATION, use_batch=USE_BATCH_NORMALIZATION,
        drop_rate=DROPOUT_RATE, num_nodes=NUM_NODES,
        use_init=USE_SPECIAL_INITIALIZATION, init=INITIALIZER, activ=ACTIVATOR,
        use_adam=USE_ADAM_OPTIMIZER, lr=ADAM_LR, b1=ADAM_BETA1, b2=ADAM_BETA2,
        use_sched=USE_LR_SCHEDULER, decay_time=LR_10FOLD_DECAY_TIME, n_epo=N_EPOCHS):
    
    model = initialize_model(use_drop, use_batch, drop_rate, num_nodes, use_init, init, activ)
    model = compile_model(model, use_adam, lr, b1, b2)
    history, model = fit_model(model, use_sched, decay_time, lr, use_adam, n_epo)
    rmse = analyze_performance(history, model)
    return(history, model, rmse)

In [8]:
run()

Train on 64863 samples
Epoch 1/30
64863/64863 [==============================] - 173s 3ms/sample - loss: 4.5575
Epoch 2/30
64863/64863 [==============================] - 122s 2ms/sample - loss: 4.5307
Epoch 3/30
64863/64863 [==============================] - 132s 2ms/sample - loss: 4.5300
Epoch 4/30
64863/64863 [==============================] - 120s 2ms/sample - loss: 4.5295
Epoch 5/30
64863/64863 [==============================] - 118s 2ms/sample - loss: 4.5290
Epoch 6/30
64863/64863 [==============================] - 119s 2ms/sample - loss: 4.5287
Epoch 7/30
64863/64863 [==============================] - 131s 2ms/sample - loss: 4.5289
Epoch 8/30
64863/64863 [==============================] - 121s 2ms/sample - loss: 4.5290
Epoch 9/30
64863/64863 [==============================] - 120s 2ms/sample - loss: 4.5284
Epoch 10/30
64863/64863 [==============================] - 122s 2ms/sample - loss: 4.5284
Epoch 11/30
64863/64863 [==============================] - 126s 2ms/sample - loss: 4.5

(<tensorflow.python.keras.callbacks.History at 0x1a4bba6f90>,
 2.323650147174013)

In [9]:
# beta(1,2) correspond to the importance of (momentum, scale(?))
# defaults are lr = 0.001, beta1 = 0.9, beta2 = 0.999
optimizer = keras.optimizers.Adam(learning_rate=ADAM_LR, beta_1=ADAM_BETA1, beta_2=ADAM_BETA2)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# adjust the learning rate
def lr_fn(epoch, init=ADAM_LR, decay_time=LR_10FOLD_DECAY_TIME):
    return(init * 10**(-epoch/decay_time))
lr_scheduler = keras.callbacks.LearningRateScheduler(lr_fn)
history = model.fit(X_train_scaled, y_train, epochs=N_EPOCHS, callbacks=[lr_scheduler])

NameError: name 'model' is not defined

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential()
if USE_DROPOUT_REGULARIZATION:
    model.add(keras.layers.Dropout(rate=DROPOUT_RATE, input_shape=X_train_scaled.shape[1:]))
model.add(keras.layers.Dense(19, activation=ACTIVATOR, kernel_initializer=INITIALIZER))
if USE_BATCH_NORMALIZATION:
    model.add(keras.layers.BatchNormalization())
for i in np.arange(19, 1, -18/NUM_NODES): # always start with 19 and end before 1
    j = int(i)
    model.add(keras.layers.Dropout(rate=DROPOUT_RATE))
    model.add(keras.layers.Dense(j, activation=ACTIVATOR, kernel_initializer=INITIALIZER))
    if USE_BATCH_NORMALIZATION:
        model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(1, kernel_initializer=INITIALIZER)) # the final, predicting layer
model.summary()

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
keras.__version__

In [ ]:
help(keras.optimizers.Adam)

In [ ]:
for i in np.arange(1, 10, 1.5):
    print(i)

In [ ]:
for i in np.arange(19, 1, -18/NUM_NODES):
    print(i)

In [ ]:
NUM_NODES